# 8. Game Playing and Adversarial Search

In [1]:
import time
import platform
import numpy as np
from math import inf
from random import choice
from tkinter import Tk, Button, Label, messagebox

## Players: Define all players in this game.
The move of HUMAN is defined as '-1'. 

The move of COMPUTER is defined as '+1'.

The initial board is a 3\*3 zero matrix. 

In [2]:
HUMAN = -1
COMPUTER = +1
board = np.zeros((3,3),dtype=int)
print('board:\n')
for item in board:
    print('\t%3s %3s %3s' % (item[0], item[1], item[2]))

board:

	  0   0   0
	  0   0   0
	  0   0   0


In [3]:
for item in board:
    print(item[1])

0
0
0


## Possible actions: Find empty cell in the current state.
This function helps to find all available moves in the current state.

In [4]:
def empty_cells(state):
    '''
        Each empty cell in state will be added into cells' list.
        Parameter:
            state: the state of the current board
        Return: a list of empty cells
    '''
    cells = []
    for x, row in enumerate(state):
        for y, cell in enumerate(row):
            if(cell == 0):
                cells.append([x, y])
    return cells

## Terminal tests: test if the human or computer wins.
Functions $\textit{wins()}$ and $game\_over()$ judge whether one player (COMPUTER or HUMAN) wins in the current state.

In [5]:
def wins(state, player):
    '''
        Judge whether player wins at present. E.g.:
            * Three rows    [X X X] or [O O O]
            * Three cols    [X X X] or [O O O]
            * Two diagonals [X X X] or [O O O]
        Parameter: 
            state: the state of the current board
            player: HUMAN or COMPUTER
        Return: True if a player wins
    '''
    win_state = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    if [player, player, player] in win_state:
        return True
    else:
        return False


def game_over(state):
    '''
        This function test if the human or computer wins.
        Parameter:
            state: the state of the current board
        Return: True if the human or computer wins
    '''
    return wins(state, HUMAN) or wins(state, COMPUTER)

## Transition model: Record the moves.
Function $transition\_board()$ update board with the moves of players.

Function $print\_board()$ shows the updated board.

In [6]:
def transition_board(x, y, player):
    board[x][y] = player
    print_board(player)


def print_board(player):
    if player == 1:
        print('COMPUTER:\n')
    elif player == -1:
        print('HUMAN:\n')
    else:
        print('Initial:\n')
    for item in board:
        print('\t %3s %3s %3s' % (item[0], item[1], item[2]))

In [7]:
for item in board:
    print(item, ' ')

[0 0 0]  
[0 0 0]  
[0 0 0]  


## Utility function: Minimax search.
Function $minimax()$ dose minimax search to choose the next move for COMPUTER.

Function $min\_utility()$ finds the minimum utility move of the opponent's possible moves.

Function $max\_utility()$ finds the maximum utility move.

In [8]:
def minimax(state, player):
    '''
        Minimax search that choices the best move.
        Parameter: 
            state: the state of the current board
            player: HUMAN or COMPUTER
        Return: the best moving coordinate and its utility, [best_row, best_col, utility].
            E.g. return [0, 1, 1] 
    '''
    best_move = [-1, -1, -inf]
    nextstate = state
    max_u = None
    
    # Find the best moving coordinate that provides the maximum utility, 
    # assuming the opponent also makes a best move.
    for [x, y] in empty_cells(state):
        
        # Make our move is [x, y].
#         = cell[0], cell[1]
        nextstate[x][y] = player
        
        # After making our move, find the best move the opponent can make (best for opponent = worse for us).
        # If we consider the opponent winning as negative utility (-1), 
        # we want to find the minimum utility move of the opponent's possible moves.
        u = min_utility(nextstate, -player)
        
        # Compare with other moves and record the best.
        if max_u is None or u > max_u:
            best_move[0] = x
            best_move[1] = y
            best_move[2] = u
            max_u = u
        
        # Reset our move.
        nextstate[x][y] = 0
    return best_move


def min_utility(state, player):
    '''
        This function finds the minimum utility move of the opponent's possible moves.
        Parameter:
            state: the state of the current board
            player: HUMAN or COMPUTER
        Return: the minimum utility
    '''
    nextstate = state
    depth = len(empty_cells(state))
    
    # If the current state is a WIN/LOSS/DRAW, stop searching.
    if depth == 0 or game_over(state):
        if wins(state, COMPUTER):
            score = +1
        elif wins(state, HUMAN):
            score = -1
        else:
            score = 0
        return [-1, -1, score]
    else:
        min_u = None
        for [x, y] in empty_cells(state):
            
            # Make a move [x, y].
            nextstate[x][y] = player
            
            # After making a move (current player is in the "player" variable), 
            # find the minimum next move and return its utility.
            u = max_utility(nextstate, -player)
            if min_u is None or u < min_u:
                min_u = u
            
            # Reset the move.
            nextstate[x][y] = 0
        return min_u


def max_utility(state, player):
    '''
        This function finds the maximum utility move.
        Parameter:
            state: the state of the current board
            player: HUMAN or COMPUTER
        Return: the maximum utility
    '''
    nextstate = state
    depth = len(empty_cells(state))
    # If the current state is a WIN/LOSS/DRAW, stop searching.
    if depth == 0 or game_over(state):
        if wins(state, COMPUTER):
            score = +1
        elif wins(state, HUMAN):
            score = -1
        else:
            score = 0
        return [-1, -1, score]
    else:
        max_u = None
        for [x, y] in empty_cells(state):
            
            # Make a move [x, y].
            nextstate[x][y] = player
            
            # After making a move (current player is in the "player" variable), 
            # find the maximum next move and return its utility.
            u = min_utility(nextstate, -player)
            if max_u is None or u > max_u:
                max_u = u
            
            # Reset the move.
            nextstate[x][y] = 0
        return max_u

## Utility function: Alpha-beta pruning
Function $minimax\_ab()$ dose minimax search with alpha-beta pruning to choose the next move for COMPUTER.

Function $min\_utility\_ab()$ finds the minimum utility move of the opponent's possible moves.

Function $max\_utility\_ab()$ finds the maximum utility move.

In [9]:
def minimax_ab(state, player):
    '''
        Minimax search with alpha-beta pruning that choices the best move.
        Parameter: 
            state: the state of the current board
            player: HUMAN or COMPUTER
        Return: the best moving coordinate and its utility, [best_row, best_col, utility].
            E.g. return [0, 1, 1] 
    '''
    best_move = [-1, -1, -inf]
    max_u = None
    alpha = None
    beta = None
    nextstate = state
    # Find the move that provides the maximum utility, assuming the opponent also makes a best move.
    for [x, y] in empty_cells(state):
        
        # Make a move [x, y].
        nextstate[x][y] = player
        
        # After making our move, find the best move the opponent can make (best for opponent = worse for us)
        # If we consider the opponent winning as negative utility (-1), 
        # we want to find the minimum utility move of the opponent's possible moves.
        u = min_utility_ab(nextstate, -player, alpha, beta)
        
        # Reset our move.
        nextstate[x][y] = 0
        
        # Compare with other moves and record the best.
        if max_u is None or u > max_u:
            best_move[0] = x
            best_move[1] = y
            best_move[2] = u
            max_u = u
        
        # If the utility we just found is greater than beta, and these utilities can only get bigger 
        # (because we're in a max stage), then there is no reason to check further.
        if beta is not None and u >= beta:
            return [x, y, u]
        
        # If the utility we just found (from the min_utility_ab stage) is greater than alpha, we found a new greatest min,
        # this will restrict future searches not to look further, if they are minimizing and find a utility less than alpha.
        if alpha is None or u > alpha:
            alpha = u

    return best_move


def min_utility_ab(state, player, alpha, beta):
    '''
        This function finds the minimum utility move of the opponent's possible moves.
        Parameter:
            state: the state of the current board
            player: HUMAN or COMPUTER
            alpha: a parameter of alpha-beta pruning algorithm
            beta: a parameter of alpha-beta pruning algorithm
        Return: the minimum utility
    '''
    nextstate = state
    depth = len(empty_cells(state))
    
    # If the current state is a WIN/LOSS/DRAW, stop searching.
    if depth == 0 or game_over(state):
        if wins(state, COMPUTER):
            score = +1
        elif wins(state, HUMAN):
            score = -1
        else:
            score = 0
        return [-1, -1, score]
    else:
        min_u = None
        for [x, y] in empty_cells(state):
            
            # Make a move [x, y].
            nextstate[x][y] = player
            
            # After making a move (current player is in the "player" variable), 
            # find the minimum next move and return its utility.
            u = max_utility_ab(nextstate, -player, alpha, beta)
            
            # Reset the move.
            nextstate[x][y] = 0
            
            # Record the minimum utility.
            if min_u is None or u < min_u:
                min_u = u
            
            # If the utility we just found is smaller than alpha, and these utilities can only get smaller 
            # (because we're in a min stage), then there is no reason to check further.
            if alpha is not None and u <= alpha:
                return u
            
            # If the utility we just found (from the max stage) is smaller than beta, 
            # we found a new smallest max; 
            # this will restrict future searches not to look further,
            # if they are maximizing and find a utility greater than beta.
            if beta is None or u < beta:
                beta = u

        return min_u


def max_utility_ab(state, player, alpha, beta):
    '''
        This function finds the maximum utility move.
        Parameter:
            state: the state of the current board
            player: HUMAN or COMPUTER
            alpha: a parameter of alpha-beta pruning algorithm
            beta: a parameter of alpha-beta pruning algorithm
        Return: the maximum utility
    '''
    nextstate = state
    depth = len(empty_cells(state))
    
    # If the current state is a WIN/LOSS/DRAW, stop searching.
    if depth == 0 or game_over(state):
        if wins(state, COMPUTER):
            score = +1
        elif wins(state, HUMAN):
            score = -1
        else:
            score = 0
        return [-1, -1, score]
    else:
        max_u = None
        for [x, y] in empty_cells(state):
            
            # Make a move [x, y].
            nextstate[x][y] = player
            
            # After making a move (current player is in the "player" variable), 
            # find the maximum next move and return its utility.
            u = min_utility_ab(nextstate, -player, alpha, beta)
            
            # Reset the move.
            nextstate[x][y] = 0
            
            # Record the minimum utility.
            if max_u is None or u > max_u:
                max_u = u
            
            # If the utility we just found is greater than beta, and these utilities can only get bigger
            # (because we're in a max stage), then there is no reason to check further.
            if beta is not None and u >= beta:
                return u
            
            # If the utility we just found (from the min stage) is greater than alpha, 
            # we found a new greatest min; 
            # this will restrict future searches not to look further, 
            # if they are minimizing and find a utility less than alpha.
            if alpha is None or u > alpha:
                alpha = u

        return max_u

## Initial: Some description of the player's starting situation.
Function $human\_turn()$ makes a move of HUMAN.

Function $ai\_turn()$ makes a move of COMPUTER.

In [10]:
def human_turn(number_cell):
    """
        HUMAN choses a valid move.
        Parameter:
            number_cell: a number of cells from 1~9
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return
    
    # Dictionary of valid moves
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }
    [x, y] = moves[number_cell]
    
    # Check whether the move is valid.
    if [x, y] in empty_cells(board):
        transition_board(x, y, HUMAN)


def ai_turn():
    """
        This function calls the minimax function if the depth < 9, else it choices a random coordinate.
        Return: the best moving coordinates
    """
    depth = len(empty_cells(board))
    
    # Check whether one side wins.
    if depth == 0 or game_over(board):
        return
    
    # If COMPUTER first, random decide one move.
    if depth == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        
        # Call minimax function to decide the next move.
        move = minimax(board, COMPUTER)
        
        # Call minimax-alpha-beta-pruning function to decide the next move.
#         move = minimax_ab(board, COMPUTER)
        
        x, y = move[0], move[1]
    
    # Check whether the move is valid.
    if [x, y] in empty_cells(board):
        transition_board(x, y, COMPUTER)
    time.sleep((10-depth)/10)
    return move

## Define buttons on GUI.
Function $define\_buttons$ defines nine buttons for a tic-tac-toe game.

In [11]:
def define_buttons():
    '''
        This function defines nine buttons for a Tic-Tac-Toe game.
    '''
    global button1, button2, button3, button4, button5, button6, button7, button8, button9, button_list
    
    button1 = Button(tk, text=' ', font='Times 20 bold', bg='Silver', fg='white', height=4, width=8, command=lambda: btnClick(button1))
    button2 = Button(tk, text=' ', font='Times 20 bold', bg='Silver', fg='white', height=4, width=8, command=lambda: btnClick(button2))
    button3 = Button(tk, text=' ', font='Times 20 bold', bg='Silver', fg='white', height=4, width=8, command=lambda: btnClick(button3))
    button4 = Button(tk, text=' ', font='Times 20 bold', bg='Silver', fg='white', height=4, width=8, command=lambda: btnClick(button4))
    button5 = Button(tk, text=' ', font='Times 20 bold', bg='Silver', fg='white', height=4, width=8, command=lambda: btnClick(button5))
    button6 = Button(tk, text=' ', font='Times 20 bold', bg='Silver', fg='white', height=4, width=8, command=lambda: btnClick(button6))
    button7 = Button(tk, text=' ', font='Times 20 bold', bg='Silver', fg='white', height=4, width=8, command=lambda: btnClick(button7))
    button8 = Button(tk, text=' ', font='Times 20 bold', bg='Silver', fg='white', height=4, width=8, command=lambda: btnClick(button8))
    button9 = Button(tk, text=' ', font='Times 20 bold', bg='Silver', fg='white', height=4, width=8, command=lambda: btnClick(button9))
    
    button1.grid(row=0, column=0)
    button2.grid(row=0, column=1)
    button3.grid(row=0, column=2)
    button4.grid(row=1, column=0)
    button5.grid(row=1, column=1)
    button6.grid(row=1, column=2)
    button7.grid(row=2, column=0)
    button8.grid(row=2, column=1)
    button9.grid(row=2, column=2)
    
    button_list = [button1, button2, button3, button4, button5, button6, button7, button8, button9]

## Define the button click event.
Function $btnClick()$ defines the call back function for a button click event.

Function $buttonToNumber()$ recieves a button and returns its corresponding number.

Function $numberToButton()$ recieves a number and returns its corresponding button.

In [12]:
def btnClick(button):
    '''
        A button click event.
    '''
    # If the button hasn't been clicked.
    if button["text"] == " ":
        # HUMAN turn. The clicked button turns to Black X.
        button["text"] = "X"
        button['bg'] = 'Black'
        human_turn(buttonToNumber(button))
        
        # Check whether the game is over.
        if len(empty_cells(board)) > 0 and not game_over(board):
            # AI turn. The best moving button turns to FireBrick O.
            coords = ai_turn()
            ai_button = numberToButton(coords)
            ai_button["text"] = "O"
            ai_button['bg'] = 'FireBrick'
    
    # If game's over, show GAME OVER and winner messages.
    if len(empty_cells(board)) < 1 or game_over(board):
        if wins(board, HUMAN):
            print('GAME OVER: \n \t HUMAN WIN')
            messagebox.showinfo("GAME OVER","HUMAN WIN")
        elif wins(board,COMPUTER):
            print('GAME OVER: \n \t COMPUTER WIN')
            messagebox.showinfo("GAME OVER","COMPUTER WIN")
        else:
            print('GAME OVER: \n \t DRAW')
            messagebox.showinfo("GAME OVER","DRAW")
        tk.destroy()


def buttonToNumber(button):
    '''
        This function transforms the button to its number.
        Parameter:
            button: the clicked button
        Return: a number of the clicked button
    '''
    return button_list.index(button) + 1


def numberToButton(coords):
    '''
        This function transforms coordinates to button.
        Parameter:
            coords: the best moving coordinates
        Return: a button
    '''
    # Dictionary of valid moves
    moves = {
        '00': 1, '01': 2, '02': 3,
        '10': 4, '11': 5, '12': 6,
        '20': 7, '21': 8, '22': 9,
    }
    return button_list[moves[str(coords[0])+str(coords[1])] - 1]

## Main function.

In [13]:
if __name__ == '__main__':
    # Print the initial board.
    print_board(0)
    
    tk = Tk()
    tk.title("Tic Tac Toe")

    define_buttons()

    messagebox.showinfo("Tips"," Human (X) \n Computer (O) \n Human first")

    tk.mainloop()

Initial:

	   0   0   0
	   0   0   0
	   0   0   0
HUMAN:

	  -1   0   0
	   0   0   0
	   0   0   0
COMPUTER:

	  -1   0   0
	   0   1   0
	   0   0   0
HUMAN:

	  -1   0   0
	  -1   1   0
	   0   0   0
COMPUTER:

	  -1   0   0
	  -1   1   0
	   1   0   0
HUMAN:

	  -1  -1   0
	  -1   1   0
	   1   0   0
COMPUTER:

	  -1  -1   1
	  -1   1   0
	   1   0   0
GAME OVER: 
 	 COMPUTER WIN
